## 3.2 운영 중인 노드에 실행 명령하기

In [19]:
import requests
import json
import pandas as pd
import hashlib      # hash 함수용 sha256 사용할 라이브러리
import random

### 1. 노드의 블록 정보 확인

In [20]:
# one_node.ipynb 실행켜야 서버에서 정보 받아옴

headers = {'Content-Type': 'application/json; charset=utf-8'}
res = requests.get("http://localhost:5000/chain", headers=headers)
json.loads(res.content)

{'chain': [{'index': 1,
   'nonce': 100,
   'previous_hash': 1,
   'timestamp': 1727089816.295097,
   'transactions': []}],
 'length': 1}

### 2-1. transaction 입력하기

In [21]:
# transaction 입력하기
headers = {'Content-Type' : 'application/json; charset=utf-8'}
data = {
    "sender": "test-from",
    "recipient": "test-to",
    "amount" : 3,
    "smart_contract": {"contract_address":"myaddress"}
}

requests.post("http://localhost:5000/transactions/new", headers=headers,data = json.dumps(data)).content

b'{"message":"Transaction will be added to Block {2}"}\n'

### 2-2. 노드의 블록 정보 확인 - 2

In [22]:
## 노드의 블록정보 확인 - 2
headers = {'Content-Type': 'application/json; charset=utf-8'}
res = requests.get("http://localhost:5000/chain", headers=headers)
json.loads(res.content)

{'chain': [{'index': 1,
   'nonce': 100,
   'previous_hash': 1,
   'timestamp': 1727089816.295097,
   'transactions': []}],
 'length': 1}

### 3-1. 채굴하기 -1

In [29]:
# 채굴하기
headers = {'Content-Type' : 'application/json; charset=utf-8'}
res = requests.get("http://localhost:5000/mine")
print(res)
print(res.text)

<Response [200]>
{"index":4,"message":"new block found","nonce":8417091,"previous_hash":"c351f8ae3529ddb9fde58bb83ae413e56d34982ed36c45370c640d2bf3e89e42","transactions":[{"amount":3,"recipient":"test-to4","sender":"test-to","timestamp":1727090098.858969},{"amount":2,"recipient":"test-to3","sender":"test-to","timestamp":1727090614.1401753},{"amount":0.1,"recipient":"node_5000","sender":"master","timestamp":1727091337.012978}]}



### 3-2. 노드의 블록 정보 확인 -3

In [30]:
headers = {'Content-Type': 'application/json; charset=utf-8'}
res = requests.get("http://localhost:5000/chain", headers=headers)
json.loads(res.content)

{'chain': [{'index': 1,
   'nonce': 100,
   'previous_hash': 1,
   'timestamp': 1727089816.295097,
   'transactions': []},
  {'index': 2,
   'nonce': 9317841,
   'previous_hash': 'e85f9a67c9d69f06eafa0089baf6f5d52dc1ffe4d1b07ccd72706ce7276cb921',
   'timestamp': 1727089826.2203434,
   'transactions': [{'amount': 3,
     'recipient': 'test-to',
     'sender': 'test-from',
     'timestamp': 1727089822.063624},
    {'amount': 0.1,
     'recipient': 'node_5000',
     'sender': 'master',
     'timestamp': 1727089826.2203434}]},
  {'index': 3,
   'nonce': 770304,
   'previous_hash': '8c28c7739040cb0f2e1bd97e95e559327db2b2156bafd83b6038992497761920',
   'timestamp': 1727089834.5278556,
   'transactions': [{'amount': 30,
     'recipient': 'test-to2',
     'sender': 'test-from',
     'timestamp': 1727089830.341018},
    {'amount': 300,
     'recipient': 'test-to3',
     'sender': 'test-from',
     'timestamp': 1727089832.3899186},
    {'amount': 0.1,
     'recipient': 'node_5000',
     'sender'

### 4. transaction2 입력하기 + 채굴 + 블록 정보 확인

In [25]:
# transaction 입력하기 - 2
headers = {'Content-Type' : 'application/json; charset=utf-8'}
data = {
    "sender": "test-from",
    "recipient": "test-to2",
    "amount" : 30,
}

requests.post("http://localhost:5000/transactions/new", headers=headers,data = json.dumps(data)).content

# transaction 입력하기 - 3
headers = {'Content-Type' : 'application/json; charset=utf-8'}
data = {
    "sender": "test-from",
    "recipient": "test-to3",
    "amount" : 300,
}

requests.post("http://localhost:5000/transactions/new", headers=headers,data = json.dumps(data)).content

# 채굴하기
headers = {'Content-Type' : 'application/json; charset=utf-8'}
res = requests.get("http://localhost:5000/mine")
print(res)

# 노드의 블록정보 확인 - 4
headers = {'Content-Type': 'application/json; charset=utf-8'}
res = requests.get("http://localhost:5000/chain", headers=headers)

<Response [200]>


In [26]:
res.text

'{"chain":[{"index":1,"nonce":100,"previous_hash":1,"timestamp":1727089816.295097,"transactions":[]},{"index":2,"nonce":9317841,"previous_hash":"e85f9a67c9d69f06eafa0089baf6f5d52dc1ffe4d1b07ccd72706ce7276cb921","timestamp":1727089826.2203434,"transactions":[{"amount":3,"recipient":"test-to","sender":"test-from","timestamp":1727089822.063624},{"amount":0.1,"recipient":"node_5000","sender":"master","timestamp":1727089826.2203434}]},{"index":3,"nonce":770304,"previous_hash":"8c28c7739040cb0f2e1bd97e95e559327db2b2156bafd83b6038992497761920","timestamp":1727089834.5278556,"transactions":[{"amount":30,"recipient":"test-to2","sender":"test-from","timestamp":1727089830.341018},{"amount":300,"recipient":"test-to3","sender":"test-from","timestamp":1727089832.3899186},{"amount":0.1,"recipient":"node_5000","sender":"master","timestamp":1727089834.5278556}]}],"length":3}\n'

### 5. Pandas를 활용한 거래내역 조회

In [27]:
status_json = json.loads(res.text)
status_json['chain']    
tx_amount_l = []
tx_sender_l = []
tx_reciv_l  = []
tx_time_l   = []

for chain_index in range(len(status_json['chain'])):
    chain_tx = status_json['chain'][chain_index]['transactions']
    for each_tx in range(len(chain_tx)):
        tx_amount_l.append(chain_tx[each_tx]['amount'])
        tx_sender_l.append(chain_tx[each_tx]['sender'])
        tx_reciv_l.append(chain_tx[each_tx]['recipient'])
        tx_time_l.append(chain_tx[each_tx]['timestamp'])

df_tx = pd.DataFrame()
df_tx['timestamp'] = tx_time_l  
df_tx['sender'] = tx_sender_l 
df_tx['recipient'] = tx_reciv_l
df_tx['amount'] = tx_amount_l   
df_tx

,timestamp,sender,recipient,amount
0,1.727090e+09,test-from,test-to,3.0
1,1.727090e+09,master,node_5000,0.1
2,1.727090e+09,test-from,test-to2,30.0
3,1.727090e+09,test-from,test-to3,300.0
4,1.727090e+09,master,node_5000,0.1


### 6. 거래내역 기반 계정별(지갑별) 잔액 조회

In [28]:
df_sended = pd.DataFrame(df_tx.groupby('sender')['amount'].sum()).reset_index()
df_sended.columns = ['user','sended_amount']
df_received= pd.DataFrame(df_tx.groupby('recipient')['amount'].sum()).reset_index()
df_received.columns = ['user','received_amount']
df_received

df_status = pd.merge(df_received,df_sended, on ='user', how=  'outer').fillna(0)
df_status['balance'] = df_status['received_amount']  - df_status['sended_amount']  
df_status

,user,received_amount,sended_amount,balance
0,master,0.0,0.2,-0.2
1,node_5000,0.2,0.0,0.2
2,test-from,0.0,333.0,-333.0
3,test-to,3.0,0.0,3.0
4,test-to2,30.0,0.0,30.0
5,test-to3,300.0,0.0,300.0
